In [1]:
import numpy as np
import math
from fractions import Fraction
from tqdm import tqdm
import multiprocessing as mp
from haversine import haversine, Unit
import matplotlib.pyplot as plt
import os
import random
import matplotlib
from scipy.sparse import lil_matrix, csr_matrix
import gc
import time
from multiprocessing import shared_memory

In [2]:
# Astronomy-related constants

TE = 24 * 3600  # One Earth sidereal day in seconds

RE = 6371e3  # Mean radius of Earth in meters

u = 3.986e14  # Standard gravitational parameter (μ = GM) in m^3/s^2

K = RE / pow(u, 1/3) * pow(2 * np.pi, 2/3)  

eps = 25 * np.pi / 180  # Maximum elevation angle for ground access, in radians (25°)

In [3]:
def get_allfile(path):
    """
    Get all files in a directory (excluding Jupyter checkpoint folders).
    """
    all_file = []
    for f in os.listdir(path):
        f_name = os.path.join(path, f)
        if "ipynb_checkpoints" not in f_name:
            all_file.append(f_name)
    return all_file


def satellite_period(h):
    """
    Calculate orbital period (in seconds) for altitude h (in meters).
    """
    a = RE + h
    T = float(2 * np.pi * pow(a**3 / u, 0.5))
    return T


def coverage_eta(T):
    """
    Calculate half ground coverage angle (in radians) based on period T.
    """
    eta = math.acos(K * math.cos(eps) / pow(T, 2 / 3)) - eps
    return eta


def approximate_ratio(a, b, precision=1e-3):
    """
    Approximate a/b as a reduced integer ratio with given precision.
    """
    if b == 0:
        raise ValueError("Denominator cannot be zero")
    ratio = Fraction(a, b).limit_denominator(int(1 / precision))
    return ratio.numerator, ratio.denominator


def alpha_gamma_to_lambda_phi(alpha, gamma, inc):
    """
    Convert orbital plane coordinates (alpha, gamma) to Earth-fixed coordinates (longitude, latitude).
    """
    phi = math.asin(math.sin(inc) * math.sin(gamma))
    temp = math.atan2(math.cos(inc) * math.sin(gamma), math.cos(gamma))
    lamb = (temp + alpha) % (2 * math.pi)
    if lamb > math.pi:
        lamb -= 2 * math.pi
    return lamb, phi


def is_cover(l_sat, p_sat, l_cell, p_cell, eta):
    """
    Determine whether a satellite at (l_sat, p_sat) covers a ground cell at (l_cell, p_cell).
    """
    lat_check = (p_sat - 2 * eta) < -np.pi/2 or (p_sat + 2 * eta) > np.pi/2
    lon_check = (l_sat - 2 * eta) < -np.pi or (l_sat + 2 * eta) > np.pi
    if lat_check or lon_check:
        d = haversine(
            (np.degrees(p_sat), np.degrees(l_sat)),
            (np.degrees(p_cell), np.degrees(l_cell)),
            unit=Unit.METERS
        )
        return d <= eta * RE
    else:
        if (p_sat - 2 * eta) <= p_cell <= (p_sat + 2 * eta) and \
           (l_sat - 2 * eta) <= l_cell <= (l_sat + 2 * eta):
            d = haversine(
                (np.degrees(p_sat), np.degrees(l_sat)),
                (np.degrees(p_cell), np.degrees(l_cell)),
                unit=Unit.METERS
            )
            return d <= eta * RE
    return False


def cal_supply_all_cell(h, beta, a0):
    """
    For a given stable satellite ground track (h, beta, a0),
    compute coverage of each time slot along its orbit.
    """
    T = satellite_period(h * 1e3)
    p, q = approximate_ratio(int(T), TE, precision=1e-3)
    eta = coverage_eta(T)
    step = 2 * eta
    inc = beta
    total_supply = []

    for i in range(int(2 * np.pi * q / step)):
        t = step * i / (2 * np.pi / T)
        a_sat = (a0 - (2 * np.pi / TE) * t) % (2 * np.pi)
        b_sat = (step * i) % (2 * np.pi)
        l_sat, p_sat = alpha_gamma_to_lambda_phi(a_sat, b_sat, inc)
        
        r_set = []
        for idx, demand in enumerate(demand_list):
            l_cell = demand['lat_lon'][1]
            p_cell = demand['lat_lon'][0]
            if is_cover(l_sat, p_sat, l_cell, p_cell, eta):
                S = S_set[idx]
                r_set.append([idx, S, demand['density']])
        
        if r_set:
            S_sum = np.sum([item[1] for item in r_set])
            factor = user_per_sat / S_sum
            r_set = [[item[0], item[1] * factor] for item in r_set]
            total_supply.append([r_set, l_sat, p_sat])
    
    return np.asarray(total_supply, dtype="object")


def recover_fulltime(data):
    """
    Aggregate full-time coverage over time slots for a set of random slot selections.
    """
    random_numbers, cover, n_length = data
    selected_cover = lil_matrix((1, n_length), dtype=np.float64)
    sat_location = []

    for t in range(time_split):
        for num in random_numbers:
            num = (num + t) % int(time_split)
            sat_location.append([cover[num][1], cover[num][2]])
            for idx, users in cover[num][0]:
                selected_cover[0, t * len(demand_list) + idx] += users
    
    return [selected_cover.tocsr(), sat_location]


def generate_baseset(args):
    """
    Build fulltime coverage for a satellite configuration.
    """
    file, random_numbers, sat_num = args
    file_param = file.split("/")[1].split("_")
    h = float(file_param[0])
    beta = float(file_param[1])
    alpha0 = float(file_param[2].split(".npy")[0])
    param = [h, beta, alpha0]

    cover = cal_supply_all_cell(h, beta, alpha0)
    fulltime_cover = recover_fulltime([random_numbers, cover, n_length])
    
    return [param, random_numbers, fulltime_cover, sat_num]


In [12]:
# === Simulation parameters ===
sat_height = 573  # Satellite altitude in km

# Path to precomputed candidate orbit coverage data
base_cover_path = f"candidate_orbits_eval1_{sat_height}/"

# Number of time slots in the coverage window (e.g. daily temporal resolution)
time_split = 308


In [13]:
# === Load demand data ===
demand_list = np.load("data/south_america_starlink_demand_99449.npy", allow_pickle=True)

# Convert lat/lon from degrees to radians
for idx in range(len(demand_list)):
    demand_list[idx]['lat_lon'][0] = np.radians(demand_list[idx]['lat_lon'][0])  # latitude
    demand_list[idx]['lat_lon'][1] = np.radians(demand_list[idx]['lat_lon'][1])  # longitude

# === Cell parameters ===
cell_size = 4  # degrees per cell
user_per_sat = 960  # total users a satellite can serve

# === Compute area of each demand cell ===
S_set = []
for idx, demand in enumerate(demand_list):
    p_cell = demand['lat_lon'][0]  # latitude in radians

    # Approximate cell height in meters (north-south)
    h = RE * np.radians(cell_size)

    # Compute arc length at top and bottom edges of the cell (east-west)
    L1 = RE * np.sin(np.pi / 2 - abs(p_cell)) * np.radians(cell_size)
    tmp = abs(p_cell - np.radians(cell_size))
    L2 = RE * np.sin(np.pi / 2 - min(tmp, np.pi / 2)) * np.radians(cell_size)

    # Special handling near the south pole (only for extreme southern cells)
    if tmp > np.pi / 2:
        h = RE * abs(p_cell + np.pi / 2)

    # Trapezoidal approximation of cell area
    S = 0.5 * h * (L1 + L2)
    S_set.append(S)


In [14]:
selected_params=np.load("data/TinyLEO_for_Latin_America.npy",allow_pickle=True)
print(len(selected_params))
files=[]
sat_nums=0
for s_tmp in selected_params:
    param_tmp,slots,satmaps,sat_num=s_tmp
    tmp=base_cover_path+str(int(param_tmp[0]))+"_"+str(param_tmp[1])+"_"+str(param_tmp[2])+".npy"
    files.append([tmp,slots,sat_num])
    sat_nums+=sat_num
print(files[0])
print(sat_nums)

1065
['candidate_orbits_eval1_573/573_0.0_3.07177948351002.npy', [76], 1]
1065


In [15]:
all_cover=[]
n_length=len(demand_list)*time_split

with mp.Pool(processes=100) as pool:
    all_cover = list(tqdm(pool.imap(generate_baseset, files), total=len(files), mininterval=10,maxinterval=20))
    pool.close()  # Prevents any more tasks from being submitted to the pool
    pool.join()  # Wait for the worker processes to exit

100%|██████████| 1065/1065 [00:25<00:00, 42.59it/s]


In [8]:
# === Initialize residual demand vector ===
# Repeat the demand density vector across all time slots
tmp_R = [demand['density'] for demand in demand_list]
ttmpR = tmp_R * int(time_split)
R = np.array(ttmpR).reshape(-1)  # Final residual vector R
del ttmpR, tmp_R  # Free memory

selected = []  # Selected satellite set (to be filled later)
num_processes = 110  # Number of parallel processes

# === Dot product utility for greedy selection ===
def compute_dot(args, show=False):
    """
    Compute the dot product between coverage matrix and residual R
    """
    R, cid = args
    global all_cover
    fulltime_cover = all_cover[cid][2][0]  # Sparse matrix (1, T*|cells|)
    dot = fulltime_cover @ R
    return [dot[0], cid]

# === Residual update after selecting one satellite ===
def update_residual(R, cid):
    """
    Update residual vector R after using coverage from satellite `cid`
    """
    global all_cover
    cover = all_cover[cid][2][0]  # Sparse matrix
    sat_num = all_cover[cid][3]   # Number of satellites in this set

    # Subtract served demand (per satellite * number of sats)
    R[cover.indices] -= (cover.data * sat_num)
    
    # Ensure non-negativity
    R = np.maximum(R, 0)
    return R

# === Find best candidate from a chunk ===
def find_max_in_chunk(chunk):
    """
    Find maximum dot product value and corresponding candidate ID in a chunk
    """
    chunk = np.array(chunk)
    max_value = np.max(chunk[:, 0])
    local_index = np.argmax(chunk[:, 0])
    max_index = chunk[local_index][1]
    return [max_value, int(max_index)]


In [16]:
# === Initialize residual demand ===
tmp_R = [demand['density'] for demand in demand_list]
ttmpR = tmp_R * int(time_split)  # Repeat over all time slots
R_2 = np.array(ttmpR).reshape(-1).copy()  # Residual vector
R_sum_2 = np.sum(R_2)
print("Initial residual:", R_sum_2)
del ttmpR, tmp_R

# === Greedy sequential selection and tracking ===
process = []  # Store [satellite count, coverage gain] for each selected set
for idx in range(len(all_cover)):
    # Apply satellite group `idx` and update residual
    R_2 = update_residual(R_2, idx)

    # Record improvement
    covered = R_sum_2 - np.sum(R_2)
    process.append([all_cover[idx][3], covered])
    R_sum_2 = np.sum(R_2)

    print("Residual after set", idx, ":", R_sum_2)

# === Final residual stats ===
print(f"Final residual non-zeros = {np.count_nonzero(R_2)}, "
      f"max = {np.max(R_2):.2f}, mean = {np.mean(R_2):.2f}")


Initial residual: 7006629.521062489
Residual after set 0 : 6976824.947982652
Residual after set 1 : 6947020.37490282
Residual after set 2 : 6917215.801822986
Residual after set 3 : 6887411.228743151
Residual after set 4 : 6857606.655663317
Residual after set 5 : 6827802.082583482
Residual after set 6 : 6797997.509503648
Residual after set 7 : 6768192.936423816
Residual after set 8 : 6738388.36334398
Residual after set 9 : 6708583.790264148
Residual after set 10 : 6678779.2171843145
Residual after set 11 : 6649270.315507767
Residual after set 12 : 6619761.413831219
Residual after set 13 : 6590252.512154672
Residual after set 14 : 6560743.610478125
Residual after set 15 : 6531234.708801577
Residual after set 16 : 6501725.807125028
Residual after set 17 : 6472216.9054484805
Residual after set 18 : 6442708.003771934
Residual after set 19 : 6413199.102095384
Residual after set 20 : 6383690.200418837
Residual after set 21 : 6354322.0817428045
Residual after set 22 : 6331237.5241761105
Residu

In [17]:
print(f"Remaining nonzero entries = {np.count_nonzero(R_2)}, Max = {np.max(R_2):.2f}, Mean = {np.mean(R_2):.2f}")

Remaining nonzero entries = 7, Max = 1.89, Mean = 0.00


In [18]:
plt_data=[0]
r_sum=sum([demand['density'] for demand in demand_list])*time_split
for item in process:
    for _ in range(item[0]):
        v=item[1]/item[0]/r_sum*100+plt_data[-1]
        plt_data.append(v)
np.save("data/network_availability_Latin_America_demand.npy",plt_data)

In [19]:
time_shift=0
total_supply=[0]*len(demand_list)
for cover in tqdm(all_cover):
    cover_csr = cover[2][0]
    sat_num = cover[3]
    total_supply += cover_csr.toarray().flatten()[len(demand_list)*time_shift:len(demand_list)*(time_shift+1)] * sat_num  

100%|██████████| 1065/1065 [00:06<00:00, 162.94it/s]


In [20]:
supply_demand_ratio=[]
for idx,demand in enumerate(demand_list):
    if demand['density']!=0:
        supply_demand_ratio.append(total_supply[idx]/demand['density'])
print(np.mean(supply_demand_ratio))
np.save("data/supply_demand_ratio_south.npy",supply_demand_ratio)

174.43454264899722


In [21]:
def FeasibilityCheck_one_slot(params):
    t,mega=params
    two_pi=2*np.pi
    total_supply=[]
    for idx,shell in tqdm(enumerate(mega)):
        T=T_list[idx]
        eta=coverage_eta(T)
        inc=inc_list[idx]
        for mid in range(shell["m"]):
            for nid in range(shell["n"]):
                r_set=[]
                a_sat=(two_pi/shell["m"]*mid-t/TE*two_pi)%two_pi
                b_sat=(two_pi/shell["n"]*nid+t/T*two_pi)%two_pi
                l_sat,p_sat=alpha_gamma_to_lambda_phi(a_sat,b_sat,inc)
                for idx,demand in enumerate(demand_list):
                    l_cell=demand['lat_lon'][1]
                    p_cell=demand['lat_lon'][0]
                    r=is_cover(l_sat,p_sat,l_cell,p_cell,eta) #用经纬度判断
                    if r:
                        S=S_set[idx]
                        r_set.append([idx,S,demand['density']])
                if len(r_set)!=0:
                    S_sum=np.sum([item[1] for item in r_set])
                    factor=user_per_sat/S_sum
                    r_set=[[item[0],item[1]*factor]for item in r_set]
                    total_supply.append(r_set)
    # check the demand
    supply=[0]*len(demand_list)
    for r_set in total_supply:
        for item in r_set:
            supply[item[0]]+=item[1]
    return supply

In [25]:
south_data=np.load("data/MegaReduce_South_America.npy",allow_pickle=True)
selected_data_south=south_data[-1][0]
T_list = np.array([satellite_period(shell["h"] * 1e3) for shell in selected_data_south])
inc_list = np.radians(np.array([shell["inc"] for shell in selected_data_south]))
total_supply_south=FeasibilityCheck_one_slot([0,selected_data_south])

demand_list=np.load("data/south_america_starlink_demand_99449.npy",allow_pickle=True)
for idx in range(len(demand_list)):
    demand_list[idx]['lat_lon'][0]=np.radians(demand_list[idx]['lat_lon'][0])
    demand_list[idx]['lat_lon'][1]=np.radians(demand_list[idx]['lat_lon'][1])

supply_demand_ratio=[]
for idx,demand in enumerate(demand_list):
    if demand['density']!=0:
        supply_demand_ratio.append(total_supply_south[idx]/demand['density'])
print(np.mean(supply_demand_ratio))
np.save("data/supply_demand_ratio_south_megareduce.npy",supply_demand_ratio)

353.84081046257586
